In [9]:
import os
import pandas as pd
from pandas import read_csv
import numpy as np
import re
from io import StringIO

In [15]:
# ---- File Management ----
# CSV Models
metricasDeepGroup = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/metricasCambioOrden.csv",delimiter=",")
# Predictions made by model from the TS
predictionsListRavel_ML = np.load("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/predList_ML.npy", allow_pickle=True)
# Real Values for each TS
realList_ML = np.load("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/realList_ML.npy", allow_pickle=True)

metricasDeepGroup
df = metricasDeepGroup.loc[metricasDeepGroup['id_serie'] == 39]
df = df.loc[df['rep'] == 0]
df = df.loc[df['output'] == 12]
df

,Unnamed: 0,id_model,id_serie,input,output,rep,sMAPE,MAE,MAPE
0,0,0,39,12,12,0,0.046000,4852.270264,0.089535
1,1,1,39,12,12,0,0.143779,13493.733330,0.246378
2,2,2,39,12,12,0,0.035665,3811.931971,0.072198


In [17]:
def metrics(pre, real):
    
    # sMAPE
    sMAPE=np.sum(np.abs(pre-real))/np.sum((pre+real))
    
    #MAE
    # Len en Real
    MAE=np.mean(np.abs(pre-real)/len(real))
        
    #MAPE
    MAPE=np.mean(np.abs((real-pre)/real))
    
    
    return [sMAPE, MAE, MAPE]
# Input: dataframe-> TS Dataframe, realValues-> Real Values for each TS, predictionValues -> Predictions made by model from the TS
# Output: List of real and predictions averages into a data frame

def averageDataListGenerator(dataframe, realValues, predictionValues):
    # List of averages
    temp_list = []
    temp_list_averages = []
    #Metrics List
    temp_metricsList = []
    #List of averages for NPY
    realNPY = []
    predNPY = []
    for i,row in dataframe.iterrows():
        print("  ")
        print("Iter #", i)
        print("  ")
        #print(row["id_serie"], row["id_model"], row["rep"], row["output"])
        df = dataframe.loc[dataframe['id_serie'] == row.loc["id_serie"]]
        df = df.loc[df['rep'] == row.loc["rep"]]
        df = df.loc[df['output'] == row.loc["output"]]
        realAve = []
        predicAve = []
        temp_list_averages = []
        
        for j,row in df.iterrows():
            print(row["id_serie"], row["rep"], row["output"], row["sMAPE"],row["MAE"],row["MAPE"])
            # Promediar arreglos 1 x 1.
            realAve.append(realValues[j])
            predicAve.append(predictionValues[j])
        
        
        realAve = calcAverages(realAve)
        predicAve = calcAverages(predicAve)
        temp_metricsList = metrics(realAve,predicAve)
        #print(metricsList)
        #Agregamos datos generales de la serie de tiempo
        temp_list_averages.append(row["id_serie"])
        temp_list_averages.append(row["rep"])
        temp_list_averages.append(row["output"])
        #Agregamos los promedios reales de la serie de tiempo
        #temp_list_averages.append(realAve)
        realNPY.append(realAve)
        #Agregamos los promedios de predicciones de la serie de tiempo
        #temp_list_averages.append(predicAve)
        predNPY.append(predNPY)
        #Agregamos las metricas calculadas con las pred y los reales de la serie de tiempo
        temp_list_averages.append(temp_metricsList[0])
        temp_list_averages.append(temp_metricsList[1])
        temp_list_averages.append(temp_metricsList[2])
        temp_list.append(temp_list_averages)
        #print("TS AVERAGES\n")
        #print(temp_list_averages)
        #print("TEMP LIST\n")
        #print(temp_list)
  
    # Create the pandas DataFrame
    df = pd.DataFrame(temp_list, columns=["id_serie","rep","output","sMAPE", "MAE", "MAPE"])
    df.to_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/ML_averages.csv", index = False, header=True)
    #np.save("real_assembly_ts.npy",np.array(realNPY))
    #np.save("pred_assembly_ts.npy",np.array(predNPY))
    return  

# Calculate Averages for a list
def calcAverages(averages):
    #print(averages)
    a = np.zeros(shape=(len(averages[0])), dtype='float32')
    
    for listAve in averages:
        for j in range(0,len(averages[0])):
            a[j] += listAve[j]
    
    return a/len(averages)
            
#averageDataListGenerator(metricasDeepGroup,realList_ML,predictionsListRavel_ML)
df = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/ML_averages.csv",delimiter=",")
df = df.drop_duplicates()
df

,id_serie,rep,output,sMAPE,MAE,MAPE
0,39.0,0.0,12.0,0.058236,41.279713,0.123670
3,39.0,0.0,3.0,0.057682,170.775590,0.124497
6,39.0,1.0,12.0,0.091236,68.179360,0.200128
9,39.0,1.0,3.0,0.040996,118.532280,0.085255
12,39.0,2.0,12.0,0.050822,36.445217,0.106171
...,...,...,...,...,...,...
35985,47993.0,0.0,3.0,0.093886,369.533000,0.207242
35988,47993.0,1.0,12.0,0.113545,91.051310,0.256746
35991,47993.0,1.0,3.0,0.100038,373.166300,0.222779
35994,47993.0,2.0,12.0,0.120109,86.446960,0.273894


In [21]:
# dataf: CSV DADO
# dataf2: CSV GEN
def averageDataListGenerator(dataf, dataf2):
    temp_df = []
    for i,row in dataf2.iterrows():
        values = []
        
        print("Iter",i)
        #print(row["id_serie"], row["rep"], row["output"],row["sMAPE"],row["MAE"],row["MAPE"])
        print("")
        df = dataf.loc[dataf['id_serie'] == row.loc["id_serie"]]
        df = df.loc[df['rep'] == row.loc["rep"]]
        df = df.loc[df['output'] == row.loc["output"]]
        df = df.groupby(['id_serie', 'output', 'rep']).mean()
        #print(df.head())
        
        #id_serie
        values.append(int(row["id_serie"]))
        #rep
        values.append(int(row["rep"]))
        #output
        values.append(int(row["output"]))
        #sMAPE
        sMAPE_df1 = float(df['sMAPE'])
        sMAPE_df2 = float(row.loc['sMAPE'])
        values.append(((sMAPE_df1 + sMAPE_df2)/2))
        #MAE
        MAE_df1 = float(df['MAE'])
        MAE_df2 = float(row.loc['MAE'])
        values.append(((MAE_df1 + MAE_df2)/2))
        #MAPE
        MAPE_df1 = float(df['MAPE'])
        MAPE_df2 = float(row.loc['MAPE'])
        values.append(((MAPE_df1 + MAPE_df2)/2))
        print(values)
        # Add to list
        temp_df.append(values)
           
    # Create the pandas DataFrame
    df = pd.DataFrame(temp_df, columns=["id_serie","rep","output","sMAPE", "MAE", "MAPE"])
    df.to_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/Final_averages_ML.csv", index = False, header=True)
    return


    
#averageDataListGenerator(metricasDeepGroup,df)
final = read_csv("C:/Users/TimeSeriesUser/Desktop/MetaLearning/ML/Final_averages_ML.csv",delimiter=",")
#final = final.drop_duplicates()
final

,id_serie,rep,output,sMAPE,MAE,MAPE
0,39,0,12,0.066692,3713.629117,0.129853
1,39,0,3,0.070670,1158.656143,0.137640
2,39,1,12,0.095272,5126.842620,0.183185
3,39,1,3,0.054450,902.593749,0.104229
4,39,2,12,0.063646,3697.056556,0.120725
...,...,...,...,...,...,...
11995,47993,0,3,0.105060,1998.804905,0.196970
11996,47993,1,12,0.118166,6617.772363,0.230015
11997,47993,1,3,0.107366,1911.881204,0.204401
11998,47993,2,12,0.124524,6337.677422,0.244423
